# Refinitiv Data Library for Python
## Delivery - OMMItemStream - Market By Price data via callback

This notebook demonstrates how to use the OMM Item Stream interface to request streaming Full Depth Orderbook data

## Import the library and load credentials

Credentials used by this and the other tutorials notebooks are stored in the **Configuration/credentials.ipynb** file.     

You should have edited the **Configuration/credentials.ipynb** to set your credentials as part of the **Quick Start** step.

In [1]:
from refinitiv.data.delivery import omm_item_stream
import datetime
import json
%run ../../Configuration/credentials.ipynb

## Open the session of your choice

Use our helper function in the Credentials notebook, **open_session(session_type)**, to create and open a session to connect to the 
- Refinitiv Data Platform directly (session_type="rdp") or via 
- Eikon 4 or Refinitiv Workspace (session_type="desktop") or via a 
- local realtime infrastructure (session_type="deployed").

You can also set a default in the **credentials** notebook

In [2]:
open_session()

# Example

## Define a function to display events

In [3]:
# Callback function to display data or status events
def display_event(eventType, event):
    currentTime = datetime.datetime.now().time()
    print("----------------------------------------------------------")
    print(">>> {} event received at {}".format(eventType, currentTime))
    # Print only 1000 characters to limit the output
    print(json.dumps(event, indent=2)[0:1000])
    return

## Create an ItemStream and register the event callbacks

In [4]:
stream = omm_item_stream.Definition(name="VOD.L", domain='MarketByPrice').get_stream()

# Refresh - the first full imaage we get back from the server
stream.on_refresh(lambda item_stream, event : display_event("Refresh", event))
# Update - as and when field values change, we receive updates from the server
stream.on_update(lambda item_stream, event : display_event("Update", event))
# Status - if data goes stale or item closes, we get a status message
stream.on_status(lambda item_stream, event : display_event("Status", event))
# Other errors
stream.on_error(lambda item_stream, event : display_event("Error", event))

## Open the ItemStream
The **open()** call to open the ItemStream is a synchronous one. This means the first event (either via on_refresh(), on_status() or on_error()) can occur before the open() method returns.   
However, if we use the **open_async()** asynchronous method instead, the first event callback will be after  open_async() returns.

In [5]:
# Library will request OrderBook from server
stream.open()
# We should intially receive the full orderbook, 
# after which we will receive updates for specific order (Add, Update, Delete)

----------------------------------------------------------
>>> Refresh event received at 16:31:42.680910
{
  "ID": 1,
  "Type": "Refresh",
  "Domain": "MarketByPrice",
  "Key": {
    "Service": "ELEKTRON_DD",
    "Name": "VOD.L"
  },
  "State": {
    "Stream": "Open",
    "Data": "Ok",
    "Text": "**All is well"
  },
  "Complete": false,
  "Qos": {
    "Timeliness": "Realtime",
    "Rate": "JitConflated"
  },
  "PartNumber": 0,
  "PermData": "AwEBJJw=",
  "SeqNumber": 59504,
  "Map": {
    "KeyType": "Buffer",
    "Summary": {
      "Fields": {
        "PROD_PERM": 249,
        "DSPLY_NAME": "VODAFONE GROUP",
        "CURRENCY": "GBp",
        "RECORDTYPE": 113,
        "RDN_EXCHD2": "LSE",
        "CONTEXT_ID": 2105,
        "DDS_DSO_ID": 12477,
        "SPS_SP_RIC": ".[SPSLSEGTP02L2",
        "ACTIV_DATE": "2021-06-16",
        "TIMACT_MS": 55900556,
        "BOOK_STATE": "N",
        "LOT_SIZE_A": 1,
        "SEQNUM": 11159371,
        "PROV_SYMB": "133215",
        "PR_RNK_RUL": "

<StreamState.Open: 3>

----------------------------------------------------------
>>> Refresh event received at 16:31:42.877872
{
  "ID": 1,
  "Type": "Refresh",
  "Domain": "MarketByPrice",
  "Key": {
    "Service": "ELEKTRON_DD",
    "Name": "VOD.L"
  },
  "State": {
    "Stream": "Open",
    "Data": "Ok",
    "Text": "**All is well"
  },
  "Complete": false,
  "Qos": {
    "Timeliness": "Realtime",
    "Rate": "JitConflated"
  },
  "ClearCache": false,
  "PartNumber": 1,
  "PermData": "AwEBJJw=",
  "SeqNumber": 59504,
  "Map": {
    "KeyType": "Buffer",
    "Entries": [
      {
        "Action": "Add",
        "Key": "MTMwLjU4QQ==",
        "Fields": {
          "ORDER_PRC": 130.58,
          "ORDER_SIDE": "ASK",
          "ACC_SIZE": 118640,
          "NO_ORD": 1,
          "LV_TIM_MS": 55835622
        }
      },
      {
        "Action": "Add",
        "Key": "MTM3QQ==",
        "Fields": {
          "ORDER_PRC": 137.0,
          "ORDER_SIDE": "ASK",
          "ACC_SIZE": 253100,
          "NO_ORD": 7,

You may notice that there are multiple Refresh events - this is often the case for the more actively traded instruments with large orders books.
Once all Refresh events have been received, you can then expect to receive Update events with just the Order changes i.e. Add, Update or Delete orders.

## Close Stream

In [6]:
stream.close()

<StreamState.Closed: 1>

### Close Session

In [7]:
close_session()